<a href="https://colab.research.google.com/github/amalks02/finwise-genai-capstone/blob/task-02-agent-tools-langgraph/agent_with_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agent_with_tools.ipynb

import os
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated, Sequence, TypedDict
from langgraph.graph.message import add_messages

# Import tools
from tools_config import calculator, python_repl, requests_tool

# --------------------
# Tool list and lookup
# --------------------
tools = [calculator, python_repl, requests_tool]
tools_by_name = {t.name: t for t in tools}

# --------------------
# Define Agent State
# --------------------
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --------------------
# Initialize Gemini LLM
# --------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

# Bind tools to LLM
model = llm.bind_tools(tools)

# --------------------
# Graph Nodes
# --------------------
def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

def call_tool(state: AgentState):
    outputs = []
    last = state["messages"][-1]
    for tool_call in getattr(last, "tool_calls", []):
        name = tool_call["name"]
        args = tool_call["args"]
        tool = tools_by_name.get(name)
        if tool:
            result = tool.invoke(args)
        else:
            result = {"error": f"Tool {name} not found"}
        outputs.append(ToolMessage(content=result, name=name, tool_call_id=tool_call["id"]))
    return {"messages": outputs}

def should_continue(state: AgentState):
    messages = state["messages"]
    if not getattr(messages[-1], "tool_calls", None):
        return "end"
    return "continue"

# --------------------
# Build workflow
# --------------------
workflow = StateGraph(AgentState)
workflow.add_node("llm", call_model)
workflow.add_node("tools", call_tool)
workflow.set_entry_point("llm")
workflow.add_conditional_edges("llm", should_continue, {"continue": "tools", "end": END})
workflow.add_edge("tools", "llm")
graph = workflow.compile()

# --------------------
# Example Queries
# --------------------

# EMI calculation
inputs_emi = {"messages": [("user", "What’s the EMI for a ₹10L loan over 5 years at 8.5% interest?")]}
for state in graph.stream(inputs_emi, stream_mode="values"):
    state["messages"][-1].pretty_print()

# Currency conversion
inputs_currency = {"messages": [("user", "Convert 5000 INR to USD using a live API.")]}
for state in graph.stream(inputs_currency, stream_mode="values"):
    state["messages"][-1].pretty_print()

# --------------------
# Interactive user input
# --------------------
query = input("Ask your question: ")
inputs_user = {"messages": [("user", query)]}
for state in graph.stream(inputs_user, stream_mode="values"):
    state["messages"][-1].pretty_print()



import os
print(os.getenv("GEMINI_API_KEY"))